# Robustness Checks for Evoluism-S
**Article**: [DOI 10.5281/zenodo.17565066](https://doi.org/10.5281/zenodo.17565066)

This notebook reproduces:
- **β_synergy = 0.70** (p < 0.01)
- **Mixed Model** (heterogeneity by country)
- **Newey-West** (time-series bias correction)

Run all cells → verify results in 60 seconds.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load panel (2001–2024, 85 countries)
panel_url = "https://raw.githubusercontent.com/Evoluit-M/Evoluism-S/data-and-analysis/data/processed/panel_evoluism_2001_2024.csv"
panel = pd.read_csv(panel_url)
print(f"Panel loaded: {panel.shape[0]} rows, {panel.shape[1]} cols")
panel.head()

## 1. Baseline OLS (β_synergy = 0.70)

In [ ]:
X = sm.add_constant(panel[['cognitive_capital', 'sleep_quality', 'age', 'gender_m']])
y = panel['innovation_output']
ols = sm.OLS(y, X).fit()
print(f"β_synergy = {ols.params['cognitive_capital']:.3f} (p = {ols.pvalues['cognitive_capital']:.3f})")
print(f"R² = {ols.rsquared:.3f}")
ols.summary()

## 2. Mixed Model (Random Effects by Country)

In [ ]:
mixed = smf.mixedlm("innovation_output ~ cognitive_capital + sleep_quality + age + gender_m",
                    panel, groups=panel["country"]).fit()
print(f"Mixed β_synergy = {mixed.params['cognitive_capital']:.3f} (p = {mixed.pvalues['cognitive_capital']:.3f})")
print(f"Var(country) = {mixed.cov_re.iloc[0,0]:.3f}")
mixed.summary()

## 3. Newey-West (HAC Standard Errors)

In [ ]:
nw = ols.get_robustcov_results(cov_type='HAC', maxlags=1)
print(f"Newey-West p-value = {nw.pvalues['cognitive_capital']:.3f}")
nw.summary()